# IC Engine Modeling for the Forklift Lab
---


Assumptions:
- isentropic compression and expansion
- Instantaneous combustion yielding equilibrium composition
- No reaction during the rest of the cycle

Numbering:
- BDC, prior compression: State 1
- TDC, after compression, before combustion: State 2
- TDC, after combustion: State 3
- BDC, after power stroke: State 4



If running this on Google Colab, uncomment the next code section and run it

In [ ]:
# !pip install cantera

In [ ]:
# Import Cantera and Numpy
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import trapz


# Check to make sure Cantera was loaded in correctly
print("Gas Constant R: {:.2f} [J kmol^-1 K^-1]".format(ct.gas_constant))


# Input parameters and geometry
Some useful equations:
$$ V_{BDC} = V_{TDC} + V_{stroke}$$
$$ \textrm{Compression Ratio} = r = \frac{V_{BDC}}{V_{TDC}}$$
$$ V_{stroke} = A_{cyl} L_{stroke} $$

In [ ]:

# Geometry parameter (bore, stroke, compression ratio
Bore = 0
Stroke = 0
r = 0

# Top Dead Center volume
VTDC = 0
# Bottom Dead Center volume
VBDC = 0

# Standard air
XO2_air = 0.21
XN2_air = 0.79

# Ideal gas constant (mole basis, J/mol.K)
Rmol = 8.314 

# Combustion chemistry model for gas composition
gas = ct.Solution('gri30.yaml')

# Mass of propane in the cylinder [kg] (use experimental data, might need a bit of thought)
mC3H8 = 0


 # State 1 - end of intake, prior to compression stroke
 Some helpful equations:

 $$ Y_i = X_i \frac{M_i}{M_{mix}}$$
 $$ M_{mix} = \sum_{i}^{N=Ns} M_i X_i$$

where $M_i$ is the molar mass of species $i$ and $M_{mix}$ is the molar mass of the mixture

Equivalence ratio:
$$ \phi = \frac{\bigg (X_{C3H8} / X_{O2}\bigg )} {\bigg (X_{C3H8} / X_{O2} \bigg)_{st}}$$


In [ ]:
# Temperature [K] (estimated)
T1 = 500

# Equivalence ratio [no unit]
phi = 0

# Gas composition [mole fraction, no unit]
XO2 = 0
XC3H8 = 0
XN2 = 0

# Store those values at the proper location
iC3H8 = gas.species_index("C3H8")
iO2 = gas.species_index("O2")
iN2 = gas.species_index("N2")

X1 = np.zeros(gas.n_species)

X1[iC3H8] = XC3H8
X1[iO2] = XO2
X1[iN2] = XN2

gas.X = X1

print(X1[gas.species_index('C3H8')])

In [ ]:



# Pressure at the end of intake stroke (requires some intermediate steps)
# Molar mass of gas mixture inside cylinder
MM_C3H8 = 0
MM_O2 = 0
MM_N2 = 0
MM_mix = 1

# Mass fractions of propane, O2 and N2
YC3H8 = 0
YO2 = 0
YN2 = 0

# Total mass of gas in cylinder
m = 0

# Density in cylinder
rho = 0

# and finally, pressure at state 1
P1 = rho * T1 * Rmol / MM_mix

print("T1: {:.2f} [K]".format(T1))
print("P1: {:.2f} [Bar]".format(P1 * 1e-5))


# State 2 - end of compression stroke, prior combustion

The composition has not changed since we are not considering any
chemistry during the compression and power strokes.

The gas object allows you to set the gas mixture to a given state, and calculate
useful thermodynamic properties. For example:
 
gas.TPX = T1, P1, X1
cp = gas.cp_mass

gives you the specific heat capacity at constant pressure of the gas 
mixture with mole fractions specified in X, at a temperature T1 and a 
pressure P1. You can replace cp by cv to get the one at constant volume,
or replace mass by mole to get the corresponding molar quantities.

Remember that the compression stroke is an isentropic process! Some useful isentropic relations for an ideal gas:
$$ \frac{P2}{P1} = \bigg(  \frac{V1}{V2} \bigg)^\gamma$$

$$ \frac{T2}{T1} = \bigg(  \frac{V1}{V2} \bigg)^{(\gamma -1 )}$$

In [ ]:
gas.TP = T1, P1
X2 = X1

# Calculate P2 and T2 using isentropic relations
cp = gas.cp_mass
cv = gas.cv_mass
gamma = cp / cv

V1 = VBDC
V2 = VTDC

P2 = P1 * (V1 / V2) ^ (gamma)
T2 = T1 * (V1 / V2) ^ (gamma - 1)


# State 3 - end of combustion process, assuming equilibrium

In [ ]:

# Set the gas to the state just before combustion
gas.TPX = T2, P2, X2

# Obtain the equilibrium composition, temperature and pressure after
# combustion using Cantera fancy tool (technically corresponds to the
# minimization of Gibbs free energy for the mixture, if that is of interest)
gas.equilibrate('UV')

# Recover temperature, pressure and composition
T3 = gas.T
P3 = gas.P
X3 = gas.X


# State 4 - end of the power stroke

In [ ]:
# Apply isentropic relations to get the state of the system after the power
# stroke.
T4 = 0
P4 = 0
X4 = X3

# Cycle analysis

$$ \eta_{therm} = \frac{W_{net}} {Q_{in}} = \frac{Q_{2-3} + Q_{4-1}} {Q_{2-3}} = 1 + \frac{ Q_{4-1}} {Q_{2-3}}$$ 

$$Q_{2-3} = m c_v (T_3 - T_2)$$


In [ ]:

eta = 0

print("Thermal Efficiency: {:.2f} %".format(eta*100))


In [ ]:
plt.rcParams.update({"font.size": 22})
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.family"] = "Times New Roman"
lw = 2.5
ms = 8

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
# ax.scatter([VBDC,VTDC, VTDC, VBDC], [P1,P2,P3,P4]*1e-5, s=100)
ax.scatter([VBDC,VTDC, VTDC, VBDC], np.array([1,2,3,4])*1.0e-5, s=100)
ax.set(xlabel="Volume [m$^3$]", ylabel="Pressure [Bar]")
ax.grid()

# Chemistry time!

In [ ]:
#########################################################################
# Input Parameters
#########################################################################

# reaction mechanism, kinetics type and compositions
reaction_mechanism = "gri30.yaml"
comp_air = "O2:1, N2:3.76"
fuel = "C3H8"

f = 1000.0 / 60.0  # engine speed [revs/s] (1000 rpm)
cr = 8.8  # compression ratio [-]
d_piston = 0.091  # piston diameter [m]
stroke = 0.086  # stroke length [m]

fuel_mass = 5e-5  # Mass of fuel injected into the cylinder [kg]

# Inlet temperature, pressure, and composition
T_inlet = 300.0  # K
p_inlet = ct.one_atm  # Pa
equivalence_ratio = 1.0

# outlet pressure
p_outlet = ct.one_atm  # Pa

# ambient properties
T_ambient = 300.0  # K
p_ambient = ct.one_atm  # Pa
comp_ambient = comp_air

# Inlet valve friction coefficient, open and close timings
inlet_valve_coeff = 1.0e-6
inlet_open = -18.0 / 180.0 * np.pi
inlet_close = 198.0 / 180.0 * np.pi

# Outlet valve friction coefficient, open and close timings
outlet_valve_coeff = 1.0e-6
outlet_open = 522.0 / 180 * np.pi
outlet_close = 18.0 / 180.0 * np.pi

# Fuel mass, spark open and close timings
spark_open = 353.0 / 180.0 * np.pi
spark_close = 354.0 / 180.0 * np.pi
spark_mass = 0.0  # kg

# Simulation time and parameters
sim_n_revolutions = 8
delta_T_max = 20.0
rtol = 1.0e-12
atol = 1.0e-16

#####################################################################
# Set up IC engine Parameters and Functions
#####################################################################

A_piston = 0.25 * np.pi * d_piston ** 2
V_stroke = A_piston * stroke
V_tdc = V_stroke / (cr - 1.0)


def crank_angle(t):
    """Convert time to crank angle"""
    return np.remainder(2 * np.pi * f * t, 4 * np.pi)


def piston_speed(t):
    """Approximate piston speed with sinusoidal velocity profile"""
    return -stroke / 2 * 2 * np.pi * f * np.sin(crank_angle(t))


#####################################################################
# Set up Reactor Network
#####################################################################
gas = ct.Solution(reaction_mechanism)


# define initial state and set up reactor
gas.TPX = T_inlet, p_inlet, comp_ambient
cyl = ct.IdealGasReactor(gas)
cyl.volume = V_tdc

# load reaction mechanism
gas = ct.Solution(reaction_mechanism)
# define inlet state
gas.TP = T_inlet, p_inlet
gas.set_equivalence_ratio(equivalence_ratio, fuel, comp_ambient)
inlet = ct.Reservoir(gas)

# inlet is modeled as a mass flow controller
inlet_mass = fuel_mass / gas.Y[gas.species_index(fuel)]

inlet_mfc = ct.MassFlowController(inlet, cyl)
inlet_delta = np.mod(inlet_close - inlet_open, 4 * np.pi)
inlet_t_open = (inlet_close - inlet_open) / 2.0 / np.pi / f
inlet_mfc.mass_flow_coeff = inlet_mass / inlet_t_open
inlet_mfc.set_time_function(
    lambda t: np.mod(crank_angle(t) - inlet_open, 4 * np.pi) < inlet_delta
)

# Spark timing
spark_delta = np.mod(spark_close - spark_open, 4 * np.pi)
spark_t_open = (spark_close - spark_open) / 2.0 / np.pi / f
spark_function = lambda t: np.mod(crank_angle(t) - spark_open, 4 * np.pi) < spark_delta

# define outlet pressure (temperature and composition don't matter)
gas.TPX = T_ambient, p_outlet, comp_ambient
outlet = ct.Reservoir(gas)

# outlet valve
outlet_valve = ct.Valve(cyl, outlet)
outlet_delta = np.mod(outlet_close - outlet_open, 4 * np.pi)
outlet_valve.valve_coeff = outlet_valve_coeff
outlet_valve.set_time_function(
    lambda t: np.mod(crank_angle(t) - outlet_open, 4 * np.pi) < outlet_delta
)

# define ambient pressure (temperature and composition don't matter)
gas.TPX = T_ambient, p_ambient, comp_ambient
ambient_air = ct.Reservoir(gas)

# piston is modeled as a moving wall
piston = ct.Wall(ambient_air, cyl)
piston.area = A_piston
piston.set_velocity(piston_speed)

# create a reactor network containing the cylinder and limit advance step
sim = ct.ReactorNet([cyl])
sim.rtol, sim.atol = rtol, atol

#####################################################################
# Run Simulation
#####################################################################

# set up output data arrays
states = ct.SolutionArray(
    cyl.thermo,
    extra=("t", "ca", "V", "m", "mdot_in", "mdot_out", "dWv_dt"),
)

# simulate with a maximum resolution of 1 deg crank angle
dt = 1.0 / (360 * f)
t_stop = sim_n_revolutions / f
while sim.time < t_stop:

    if spark_function(sim.time):
        U1 = cyl.thermo.cp_mass * cyl.mass * cyl.thermo.T

        print("Sparking now...", sim.time, crank_angle(sim.time) * 180 / np.pi)
        combustor = ct.Solution(reaction_mechanism)
        combustor.TPY = cyl.thermo.TPY
        combustor.equilibrate("UV")
        cyl.insert(combustor)
        sim.initialize()
        U2 = cyl.thermo.cp_mass * cyl.mass * cyl.thermo.T

        sim.advance(sim.time + dt)

    else:
        # perform time integration
        sim.advance(sim.time + dt)

    # calculate results to be stored
    dWv_dt = -(cyl.thermo.P - ambient_air.thermo.P) * A_piston * piston_speed(sim.time)

    # append output data
    states.append(
        cyl.thermo.state,
        t=sim.time,
        ca=crank_angle(sim.time),
        V=cyl.volume,
        m=cyl.mass,
        mdot_in=inlet_mfc.mass_flow_rate,
        mdot_out=outlet_valve.mass_flow_rate,
        dWv_dt=dWv_dt,
    )



#######################################################################
# Plot Results in matplotlib
#######################################################################


def ca_ticks(t):
    """Helper function converts time to rounded crank angle."""
    return np.round(crank_angle(t) * 180 / np.pi, decimals=1)


t = states.t

# pressure and temperature
xticks = np.arange(0, t[-1] + 0.06, 0.06)
fig, ax = plt.subplots(nrows=2, sharex=True)
ax[0].plot(t, states.P / 1.0e5)
ax[0].set_ylabel("$p$ [bar]")
ax[0].set_xlabel(r"$\phi$ [deg]")
ax[0].set_xticklabels([])
ax[1].plot(t, states.T)
ax[1].set_ylabel("$T$ [K]")
ax[1].set_xlabel(r"$\phi$ [deg]")
ax[1].set_xticks(xticks)
ax[1].set_xticklabels(ca_ticks(xticks))
# plt.show()

# p-V diagram
fig, ax = plt.subplots()
ax.plot(states.V[t > 3 * t[-1] / 4] * 1000, states.P[t > 3 * t[-1] / 4] / 1.0e5)
ax.set_xlabel("$V$ [l]")
ax.set_ylabel("$p$ [bar]")
# plt.show()

# T-S diagram
fig, ax = plt.subplots()
ax.plot(
    states.m[t > 3 * t[-1] / 4] * states.s[t > 3 * t[-1] / 4],
    states.T[t > 3 * t[-1] / 4],
)
ax.set_xlabel("$S$ [J/K]")
ax.set_ylabel("$T$ [K]")
# plt.show()

# heat of reaction and expansion work
fig, ax = plt.subplots()
# ax.plot(t, 1.0e-3 * states.heat_release_rate * states.V, label=r"$\dot{Q}$")
ax.plot(t, 1.0e-3 * states.dWv_dt, label=r"$\dot{W}_v$")
# ax.set_ylim(-1e2, 1e3)
ax.legend(loc=0)
ax.set_ylabel("[kW]")
ax.set_xlabel(r"$\phi$ [deg]")
ax.set_xticks(xticks)
ax.set_xticklabels(ca_ticks(xticks))
# plt.show()

# gas composition
fig, ax = plt.subplots()
ax.plot(t, states("O2").X, label="O2")
ax.plot(t, states("CO2").X, label="CO2")
ax.plot(t, states("CO").X, label="CO")
ax.plot(t, states("NO").X, label="NO")
ax.plot(t, states(fuel).X, label=fuel)
ax.legend(loc=0)
ax.set_ylabel("$X_i$ [-]")
ax.set_xlabel(r"$\phi$ [deg]")
ax.set_xticks(xticks)
ax.set_xticklabels(ca_ticks(xticks))

######################################################################
# Integral Results
######################################################################

# heat release
# Q = trapz(states.heat_release_rate * states.V, t)
Q = (U2 - U1) * sim_n_revolutions / 2.0
output_str = "{:45s}{:>4.1f} {}"
print(
    output_str.format(
        "Heat release rate per cylinder (estimate):", Q / t[-1] / 1000.0, "kW"
    )
)

# expansion power
W = trapz(states.dWv_dt, t)
print(
    output_str.format(
        "Expansion power per cylinder (estimate):", W / t[-1] / 1000.0, "kW"
    )
)

# Efficiency power
print(output_str.format("Efficiency (estimate):", W / Q * 100, "%\n"))

# CO emissions
MW = states.mean_molecular_weight
CO_emission = trapz(MW * states.mdot_out * states("CO").X[:, 0], t)
CO_emission /= trapz(MW * states.mdot_out, t)

CO2_emission = trapz(MW * states.mdot_out * states("CO2").X[:, 0], t)
CO2_emission /= trapz(MW * states.mdot_out, t)

NO_emission = trapz(MW * states.mdot_out * states("NO").X[:, 0], t) / trapz(
    MW * states.mdot_out, t
)
NO2_emission = trapz(MW * states.mdot_out * states("NO2").X[:, 0], t) / trapz(
    MW * states.mdot_out, t
)


print(output_str.format("CO emission (estimate):", CO_emission * 1.0e6, "ppm"))
print(output_str.format("CO2 emission (estimate):", CO2_emission * 1.0e6, "ppm"))
print(output_str.format("NO emission (estimate):", NO_emission * 1.0e6, "ppm"))
print(output_str.format("NO2 emission (estimate):", NO2_emission * 1.0e6, "ppm"))



plt.show()